In [1]:
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import warnings
from models.interactive_parameter import InteractiveUtils

warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'

**Generalised logistic function (GLF)**

![y = A + \frac{K - A}{\left( C + Q \cdot e^{-By} \right)^{ 1 / \nu} }](assets/GLF.png)

taken from https://en.wikipedia.org/wiki/Generalised_logistic_function

**Inverse GLF**
The following equation is the one used in this simulation.

![y = \frac{1}{B} \cdot ln \left( \frac{ \left( \frac{K - A}{x - A} \right)^\nu -C }{Q} \right) + E](assets/GLF_inverse.png)

A bunch of parameter can be removed, as they are not used:

- `A` is the minimum value of the x axis, which will always be `0`
- `C` and `Q` are form parameters that are meant to be set to `1` by default

With those parameters replaced by their default values, here is the simplified equation used:

![y = \frac{1}{B} \cdot ln \left( \left( \frac{K}{x} \right)^\nu -1 \right) + E](assets/GLF_inverse_simplified.png)

This equation can be generalised in other dimensions as follow:

![](assets/GLF_inverse_simplified_generalised.png)


In [2]:
def equation_single(X, E, B, K, nu):
    try:
        inner = np.power((K/ X), nu) - 1
        output: np.ndarray = (1/B) * np.log(inner) + E
    except ZeroDivisionError:
        output: np.ndarray = np.zeros_like(X)*np.nan

    return output

def equation_multi(X, Y, E, B1, K1, nu1, B2, K2, nu2):
    try:
        inner1 = np.power((K1/ X), nu1) - 1
        inner2 = np.power((K2/ Y), nu2) - 1

        output: np.ndarray = (1/B1) * np.log(inner1) + (1/B2) * np.log(inner2) + E
    except ZeroDivisionError:
        output: np.ndarray = np.zeros_like(X)*np.nan

    return output

def sigmoid_plot_single(ax, x_range, y_range, x, y, color, label):
    ax.plot(x_range, y_range, label='X', color=color)
    ax.vlines(x, 0, y, color='k', linestyle='dashed')
    ax.hlines(y, -0.1, x, color='k', linestyle='dashed')
    # ax.add_patch(Rectangle((K1, 0), 1-K1, 15, color='gray', alpha=0.2))
    
    ax.text(0.5, 0.95, 
        f'APR: {y:.2f}%', 
        transform=ax.transAxes, 
        horizontalalignment='center', 
        bbox=dict(facecolor='red', alpha=0.5))
    
    ax.set_xlabel(label)
    ax.set_ylabel('APR (%)')

    ax.set_xlim(-.1, 1.1)
    ax.set_ylim(0, 15)
    ax.grid()

def sigmoid_plot_multi(ax, grid, T, x, y, z, alpha, theta):
    ax.plot_surface(*grid, T, linewidth=0.5, cmap=cm.viridis, rstride=10, cstride=10)
    ax.contour(*grid, T, zdir='x', offset=-.5, cmap='coolwarm')
    ax.contour(*grid, T, zdir='y', offset=-.5, cmap='coolwarm')
    ax.plot([x]*2, [y]*2, [0, z], color='k', linestyle='dashed')
    ax.plot([-.5, x], [-.5, y], [z]*2, color='k', linestyle='dashed')

    ax.text2D(0.5, 0.95, 
            f'APR: {z:.2f}%', 
            transform=ax.transAxes, 
            horizontalalignment='center', 
            bbox=dict(facecolor='red', alpha=0.5))
    
    ax.set_xlabel('Economic Security')
    ax.set_ylabel(f'Network size')
    ax.set_zlabel('APR (%)')

    ax.set_xlim(-.5, 1)
    ax.set_ylim(-.5, 1)
    ax.set_zlim(0, 15)

    ax.set_xticks(np.arange(0, 1, 0.2))
    ax.set_yticks(np.arange(0, 1, 0.2))
    ax.view_init(alpha, theta)
    ax.set_box_aspect(aspect=None, zoom=0.9)

def plot_static(alpha, theta, E, B1, K1, nu1, B2, K2, nu2, node_count, max_node_count, staked_tokens, max_token_supply):
    ranges = [
        np.arange(0, 1, 0.003),
        np.arange(0, 1, 0.003)
    ]
    grid = np.meshgrid(*ranges)
    fig = plt.figure(figsize=(15, 6))

    axes = [
        fig.add_subplot(1,4,(1,2), projection='3d'),
        fig.add_subplot(1,4,3),
        fig.add_subplot(1,4,4)
    ]

    token_ratio = staked_tokens / max_token_supply
    node_ratio = node_count / max_node_count

    # 3D plot 
    sigmoid_plot_multi(axes[0], grid, 
                       equation_multi(*grid, E, B1, K1, nu1, B2, K2, nu2),
                       token_ratio, node_ratio, 
                       equation_multi(token_ratio, node_ratio, E, B1, K1, nu1, B2, K2, nu2),
                       alpha, theta)

    # 2D plot
    sigmoid_plot_single(axes[1], ranges[0], 
                        equation_single(ranges[0], E, B1, K1, nu1), 
                        token_ratio, 
                        equation_single(token_ratio, E, B1, K1, nu1), 
                        'blue', 'Economic security')

    sigmoid_plot_single(axes[2], ranges[1],
                        equation_single(ranges[1], E, B2, K2, nu2), 
                        node_ratio,
                        equation_single(node_ratio, E, B2, K2, nu2), 
                        'red', f'Network size (max: {max_node_count})')

    plt.subplots_adjust(wspace=0.2)

### Choice of base parameter for APR calculation

Although the APR could be a variable of many network parameters, those which makes most sense to use are the following ones.

**Economic security ratio**

This ratio is calculated as the value of token staked in the network divided by the market capitalisation. If the staking ratio increases, the APR decreases.
It's a gamable paramter: if a whale comes in the network and put a huge stake in his nodes' safe(s), he would increase this ratio drastically, thus would reduce the overall APR.

**Network size**

Eventhough we want the network to be as big as possible at somepoint, we don't want that this amount grows to fast. If the network size increases, the APR decreases. This is already a side effect of the current fix-budget, and should be kept in this new incentive scheme.
The network size is converted to a ratio, based on some target network size. The APR decreases with the network size increasing. This parameter prevents whales spinning up many nodes to get maximum rewards from their stake.


In [3]:
params = InteractiveUtils.loadSimulationConfigFile('assets/sigmoid.config.yaml')
InteractiveUtils.plotInteractive(plot_static, params)

Output()